In [19]:
import requests
from requests.exceptions import HTTPError
import json
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
import datetime
import re

url_main = 'http://statour.ru/ltb/'
#url_main = 'http://127.0.0.1:8000/ltb/'


while True:
    response = requests.get(url_main)
    time.sleep(1)
    if response.text != "":
        res_json = response.json()
        print(res_json)
        self_id = res_json["self_id"]
        date_of_departure = res_json["date_of_departure"]
        destination_resort = res_json["destination_resort"]
        hotel_name = res_json["hotel_name"]
        count_of_hotel_stars = res_json["count_of_hotel_stars"]
        hotel_rating = res_json["hotel_rating"]
        type_of_food = res_json["type_of_food"]
        return_date = res_json["return_date"]
        number_of_adults = res_json["number_of_adults"]
        number_of_children = res_json["number_of_children"]
        number_of_infants = res_json["number_of_infants"]
        accommodation_at_the_hotel_only = res_json["accommodation_at_the_hotel_only"]

       
        # Инициализация запроса ЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖ
        booking_name_hotel = ''
        booking_count_of_hotel_stars = ''
        booking_adress_hotel = ''
        booking_hotel_rating = ''
        booking_type_of_food = 'Питание не включено в стоимость'
        booking_type_of_room = []
        booking_price = ''
        booking_price_taxes = ''

        url = 'https://www.booking.com/searchresults.ru.html'
        if hotel_name != 'None':
            url += ("?ss=" + destination_resort.replace(' ', '+') + "%2C+" + hotel_name.replace(' ', '+'))
        else:
            url += ("?ss=" + destination_resort.replace(' ', '+'))
        new_date_of_departure = datetime.datetime.strptime(date_of_departure, '%Y-%m-%d')    
        url += ("&checkin_year=" + str(new_date_of_departure.year))
        url += ("&checkin_month=" + str(new_date_of_departure.month))
        url += ("&checkin_monthday=" + str(new_date_of_departure.day))
        new_return_date = datetime.datetime.strptime(return_date, '%Y-%m-%d')
        url += ("&checkout_year=" + str(new_return_date.year))
        url += ("&checkout_month=" + str(new_return_date.month))
        url += ("&checkout_monthday=" + str(new_return_date.day))
        url += ("&group_adults=" + str(number_of_adults))
        url += ("&group_children=" + str(int(number_of_children) + int(number_of_infants)))

        if hotel_name == 'None':
            url += ("&no_dorms=1")
            url += ("&order=price")
            if int(count_of_hotel_stars) or int(type_of_food) or int(hotel_rating) or accommodation_at_the_hotel_only:
                url += "&nflt=rpt%3D1%3B"
                if int(count_of_hotel_stars):
                    for i in range(int(count_of_hotel_stars), 6):
                        url += ("class%3D" + str(i) + "%3B")
                if int(type_of_food):
                    if str(type_of_food) == '2':
                        url += ("mealplan%3D" + "1" + "%3B")
                        url += ("mealplan%3D" + "9" + "%3B")
                        url += ("mealplan%3D" + "3" + "%3B")
                        url += ("mealplan%3D" + "4" + "%3B")
                    if str(type_of_food) == '3':
                        url += ("mealplan%3D" + "9" + "%3B")
                        url += ("mealplan%3D" + "3" + "%3B")
                        url += ("mealplan%3D" + "4" + "%3B")
                    if str(type_of_food) == '4':
                        url += ("mealplan%3D" + "3" + "%3B")
                        url += ("mealplan%3D" + "4" + "%3B")
                    if str(type_of_food) == '5':
                        url += ("mealplan%3D" + "4" + "%3B")
                if int(hotel_rating):
                    url += ("review_score%3D" + str(int(hotel_rating) * 10) + "%3B")
                if accommodation_at_the_hotel_only:
                    url += ("ht_id%3D204%3B")

        print(url)

        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument("--incognito")
        chrome_options.add_argument('--headless')
        driver = webdriver.Chrome(options=chrome_options, executable_path="C:/PyThon/chromedriver.exe")
        driver.get(url)
        # time.sleep(20)

        html = driver.page_source
        # time.sleep(5)
        # print(driver.page_source)
        new_html = html[html.find("sr-hotel__name"):html.find('</a>\n</div>\n</div>\n</div>\n</div>\n</div>')]
        try:
            stars = new_html.split('<span class="invisible_spoken">')[2].split("</span>")[0]
            if len(stars) > 30:
                booking_count_of_hotel_stars = 'Информации о звездах нет. Только рейтинг, ' \
                                               'выставленый самим вариантом размещения.'
            else:
                booking_count_of_hotel_stars = stars
        except Exception:
            print('ОШИБКА - Сведения о звездности отеля не найдены')
        try:
            booking_name_hotel = new_html.split("\n</span>")[0].split('">\n')[1]
        except Exception:
            print('ОШИБКА - Сведения о названии отеля не найдены')
        try:
            booking_adress_hotel = re.compile(r'<[^>]+>').sub('', new_html.split(
                '<div class="sr_card_address_line">')[1].split("</div>")[0]).replace(
                '\n\n\n\n\n\n', ';').replace('Показать на карте', '').replace(
                '\n\n\n', ';').replace('\n', '').replace(';;', ';').replace(';', '; ')[1:]
        except Exception:
            print('ОШИБКА - Сведения об адресе отеля не найдены')
        try:
            booking_hotel_rating = new_html.split('aria-label="')[1].split('">')[0]
        except Exception:
            print('ОШИБКА - Сведения о рейтинге отеля не найдены')
        try:
            if new_html.split('\n</h4>')[0].split(">\n")[-1]:
                booking_type_of_room.append(new_html.split('\n</h4>')[0].split(">\n")[-1] + ':')
            for i in range(1, len(new_html.split('<div class="roomNameInner">'))):
                booking_type_of_room.append(re.compile(r'<[^>]+>').sub('', new_html.split(
                    '<div class="roomNameInner">')[i].split("\n</span>")[0].replace(
                    '&nbsp;', ' ')).replace('\n', '').replace(' — ', ''))
        except Exception:
            print('ОШИБКА - Сведения о типе номера отеля не найдены')
        try:
            booking_price = new_html.split('<span class="bui-u-sr-only">\nЦена\n')[-1].split(
                '&nbsp;руб.\n</span>')[0].replace(' ', '')
        except Exception:
            print('ОШИБКА - Сведения о стоимости отеля не найдены')
        try:
            booking_price_taxes = new_html.split('prd-taxes-and-fees-under-price')[-1].split('</div>')[0].split('> ')[1]
            try:
                booking_price_taxes2 = booking_price_taxes.replace(' ', '').split('сборы(')[1].split('&nbsp;руб.')[0]
            except Exception:
                print('Сумма налога не найдена')
            else:
                booking_price_taxes = int(booking_price_taxes2)
        except Exception:
            print('ОШИБКА - Сведения о налогах отеля не найдены')
        try:
            food_html = html[html.find("sr_item_default"):html.find(booking_name_hotel)]
            try:
                booking_type_of_food = food_html.split('add-red-tag__amount--small">\n')[1].split("\n<span")[0]
            except Exception:
                print('Сведения о типе питания не найдены')
        except Exception:
            print('ОШИБКА - Сведения о питании отеля не найдены')
            
        if type(booking_price_taxes) is int:
            rezult_with_tax = str(int(booking_price) + int(booking_price_taxes))
        else:
            rezult_with_tax = str(booking_price)     
        
        
        rezult = {"self_id": self_id,
                  "destination_resort": destination_resort,
                  "booking_name_hotel": booking_name_hotel,
                  "booking_count_of_hotel_stars": booking_count_of_hotel_stars,
                  "booking_adress_hotel": booking_adress_hotel,
                  "booking_hotel_rating": booking_hotel_rating,
                  "booking_type_of_food": booking_type_of_food,
                  "booking_type_of_room": booking_type_of_room,
                  "booking_price": booking_price,
                  "booking_price_taxes": str(booking_price_taxes),
                  "rezult_with_tax": rezult_with_tax,
                 }

        requests.post('http://statour.ru/ltb/', json=rezult)
        #requests.post('http://127.0.0.1:8000/ltb/', json=rezult)
        driver.quit()
        print("FINISH ltb", self_id)
        print(rezult)
        print("#######################################################################################################")
    else:
        time.sleep(8)


{'self_id': 6, 'departure_country': 'RU', 'departure_city': 'Москва', 'date_of_departure': '2020-09-01', 'changing_the_departure_date': 'var_day_0', 'destination_country': 'Таиланд', 'destination_resort': 'Пхукет', 'hotel_name': 'None', 'count_of_hotel_stars': '3', 'hotel_rating': '0', 'type_of_food': '0', 'return_date': '2020-09-10', 'changing_return_date': 'var_day_0', 'number_of_adults': '2', 'number_of_children': '0', 'number_of_infants': '0', 'direct_flights_only': 'False', 'accommodation_at_the_hotel_only': 'False'}
https://www.booking.com/searchresults.ru.html?ss=Пхукет&checkin_year=2020&checkin_month=9&checkin_monthday=1&checkout_year=2020&checkout_month=9&checkout_monthday=10&group_adults=2&group_children=0&no_dorms=1&order=price&nflt=rpt%3D1%3Bclass%3D3%3Bclass%3D4%3Bclass%3D5%3Bht_id%3D204%3B
Сумма налога не найдена
Сведения о типе питания не найдены
FINISH ltb 6
{'self_id': 6, 'destination_resort': 'Пхукет', 'booking_name_hotel': 'OYO 610 Yamin Seaview Hotel', 'booking_coun

{'self_id': 9, 'departure_country': 'RU', 'departure_city': 'Москва', 'date_of_departure': '2020-09-02', 'changing_the_departure_date': 'var_day_0', 'destination_country': 'Таиланд', 'destination_resort': 'Пхукет', 'hotel_name': 'None', 'count_of_hotel_stars': '4', 'hotel_rating': '0', 'type_of_food': '0', 'return_date': '2020-09-11', 'changing_return_date': 'var_day_0', 'number_of_adults': '2', 'number_of_children': '0', 'number_of_infants': '0', 'direct_flights_only': 'False', 'accommodation_at_the_hotel_only': 'False'}
https://www.booking.com/searchresults.ru.html?ss=Пхукет&checkin_year=2020&checkin_month=9&checkin_monthday=2&checkout_year=2020&checkout_month=9&checkout_monthday=11&group_adults=2&group_children=0&no_dorms=1&order=price&nflt=rpt%3D1%3Bclass%3D4%3Bclass%3D5%3Bht_id%3D204%3B
Сумма налога не найдена
Сведения о типе питания не найдены
FINISH ltb 9
{'self_id': 9, 'destination_resort': 'Пхукет', 'booking_name_hotel': 'OYO 610 Yamin Seaview Hotel', 'booking_count_of_hotel_s

ConnectionError: HTTPConnectionPool(host='statour.ru', port=80): Max retries exceeded with url: /ltb/ (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x01371F88>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [ ]:
rezult